In [1]:
import os
import json
import pandas as pd
import traceback
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from vertexai import generative_models
from vertexai.generative_models import GenerativeModel, GenerationConfig
from langchain_google_genai import ChatGoogleGenerativeAI


In [2]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
KEY=os.getenv("GOOGLE_API_KEY")
print('KEY')

KEY


In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key='GOOGLE_API_KEY',temperature=0.5)  



In [2]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [3]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [4]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [8]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [9]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [10]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [11]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [12]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)

In [13]:
file_path=r"D:\mcqgenetaror\data.txt"

In [15]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [16]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [17]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [18]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [19]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking
import time
from vertexai import generative_models
from vertexai.generative_models import GenerativeModel, GenerationConfig

def get_gemini_usage(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        
        # Initialize the model
        model = GenerativeModel("gemini-pro")
        
        # Create a chat session
        chat = model.start_chat()
        
        # Call the original function
        result = func(chat, *args, **kwargs)
        
        # Calculate usage metrics
        end_time = time.time()
        execution_time = end_time - start_time
        
        # Print usage metrics
        print(f"Execution time: {execution_time:.2f} seconds")
        
        return result
    return wrapper

@get_gemini_usage
def generate_evaluate_chain(chat, text, number, subject, tone, response_json):
    prompt = f"""
    Text: {text}
    Number: {number}
    Subject: {subject}
    Tone: {tone}
    Response JSON: {response_json}
    
    Please generate and evaluate based on the above information.
    """
    
    response = chat.send_message(prompt)
    return response.text

# Usage
TEXT = "Sample text"
NUMBER = 42
SUBJECT = "Sample subject"
TONE = "Formal"
RESPONSE_JSON = {"key": "value"}

response = generate_evaluate_chain(TEXT, NUMBER, SUBJECT, TONE, str(RESPONSE_JSON))
print(response)


GoogleAuthError: Unable to find your project. Please provide a project ID by:
- Passing a constructor argument
- Using vertexai.init()
- Setting project using 'gcloud config set project my-project'
- Setting a GCP environment variable
- To create a Google Cloud project, please follow guidance at https://developers.google.com/workspace/guides/create-project